In [2]:
import sys
sys.path.append('../')
sys.path.append('../Nets/')
from glob import glob
from os.path import join, isfile, basename
from multiprocessing import Pool
import pickle
from scipy.ndimage.interpolation import rotate
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm
from keras.callbacks import Callback
from functools import partial
import seaborn as sns
import time
from NoduleClfOld import *
# from CancerClf import *
import pandas as pd
import csv
from paths import *
%pylab inline

%load_ext autoreload
%autoreload 1

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [3]:
sns.set_style('white')
data = pd.read_csv(join(PATH['DATA_OUT'], 'DATAFRAMES', 'top_patches'))
data['patientid'] = data.patchid.apply(lambda x: x[:32])
train = data.merge(pd.read_csv(PATH['LABELS']), 
                   left_on='patientid', 
                   right_on='id').drop(['patientid'], axis=1)


In [ ]:
rint = random.randint(0, len(data))
patch = load(join(PATH['ENHANCED_CROPPED'], data.patchid.values[rint]))
print(data.patchid[rint])
imshow(patch[32]);
axis('off');

In [61]:
train = train.groupby('id', sort=False)#.drop('id', axis=1)
# train.patchid = train.patchid.apply(lambda x: join(PATH['ENHANCED_CROPPED'], x))

In [ ]:
 32

In [3]:
CPU = 24
SHIFT = 5
TRAIN_VAL_RAIT = .3
OVERSAMPLING = 30
FULL_CHANNELS_N = 10
UNDERSAMPLING = 3
LOWER = -1000
UPPER = 400
BATCH_SIZE = 32
SHAPE = (18, 48, 48)

In [133]:
def augment_patch(patch, shape, shift):
    center = array(patch.shape) // 2
    
    shift = random.randint(-shift, shift, 3)
    center += shift

    rotate = random.randint(0, 2)
    patch = swapaxes(patch, 0, 2)
    patch = swapaxes(patch, 1, 0)
    # x y z
    patch = rot90(patch, k = 2 * rotate)

    patch = swapaxes(patch, 0, 2)
    patch = swapaxes(patch, 1, 2)
    
    
    left = array(shape) // 2
    right = array(shape) - left
    
    patch = [patch[center[0] - left[(0 + i) % 3]:center[0] + right[(0 + i) % 3], 
                   center[1] - left[(1 + i) % 3]:center[1] + right[(1 + i) % 3], 
                   center[2] - left[(2 + i) % 3]:center[2] + right[(2 + i) % 3]] for i in range(3)]
    
    patch[1] = swapaxes(patch[1], 0, 2)
    patch[2] = swapaxes(patch[2], 0, 1)
    patch = [expand_dims(array(patch[i]), -1)  
             for i in range(3)]
    patch = clip(patch, LOWER, UPPER) 
    patch = (patch - LOWER) / float(UPPER - LOWER)         
    
    return patch

In [134]:
def batch_generator(data, 
                    shape=(18, 48, 48), 
                    shift=SHIFT, 
                    CPU=24,
                    oversampling=OVERSAMPLING):
    
    with Pool(CPU) as pool:
        patch_list = pool.map(load, data) 

    augment = partial(augment_patch, 
                      shape=shape, 
                      shift=shift)
    
    oversampled = list()
    for i in range(oversampling):
        with Pool(CPU) as pool:
            augmented = pool.map(augment, patch_list)
    
        oversampled.append([patch for patch in swapaxes(asarray(augmented), 0, 1)])
    return oversampled

In [135]:
# msk = random.rand(len(train)) < 0.8
# valid = train[msk]
# train = train[msk]
# train.to_csv(join(PATH['DATA_OUT'], 'DATAFRAMES', 'train_cancer'))
# valid.to_csv(join(PATH['DATA_OUT'], 'DATAFRAMES', 'valid_cancer'))
train = pd.read_csv(join(PATH['DATA_OUT'], 'DATAFRAMES', 'train_cancer'))
valid = pd.read_csv(join(PATH['DATA_OUT'], 'DATAFRAMES', 'valid_cancer'))

In [136]:
clf_model, coders_model, bottle_neck = predictor(dropout_conv=.2, 
                                                 dropout_dence=.3, 
                                                 shape=(18, 48, 48, 1),
                                                 shared_layers=True
                                                )

clf_model.compile('adam', 'categorical_crossentropy')
clf_model.load_weights(join(PATH['WEIGHTS'], '3DCNN_nodule_clf_shared_tested'))

In [142]:
candidates = dict()
for name, group in train:
    candidates[name] = group

In [4]:
path = '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/KAGGLE_CANDIDATES/'

In [17]:
for patient, data in tqdm(candidates.items()):
    paths = list()
    for candidate in data.patchid:
        paths.append(join(PATH['ENHANCED_CROPPED'], candidate))
    
    
    batches = batch_generator(paths)
    predicted = list()
    for batch in batches:
        predicted.append(bottle_neck.predict(batch))
        
    fold = 'BENING'
    if data.cancer.values[0]:
        fold = 'CANCER'
    break
    for i, pred in enumerate(predicted):
        save(join(path, fold, patient + '_BN_' + str(i)), pred[0])
        save(join(path, fold, patient + '_DENCE_' + str(i)), pred[1])

NameError: name 'candidates' is not defined

In [5]:
def apply(layers, tnsr):
    for layer in layers:
        tnsr = layer(tnsr)
    return tnsr

In [6]:
from keras.regularizers import l1, l2

def cancer_clf(input_chanels=10,
               shape=(4, 3, 3, 256),
               dropout_conv=.2,
               dropout_dence=.2):
   
    # Determine proper input shape
    if K.image_dim_ordering() != 'tf':
        print('Wrong dim ordering: should be TF')
        
    inputs = [Input(shape=shape) 
              for i in range(input_chanels)]
    
    x =  merge(inputs, mode='ave')
    x = Convolution3D(1024, 3, 3, 3, 
                      border_mode='same',
                      W_regularizer=l2(0.01)
#                       activity_regularizer=l1(0.01)
                     )(x)     
    x = BatchNormalization(axis=4)(x)
    x = Activation('relu')(x)
    x = AveragePooling3D((2, 2, 2))(x)
        
    if dropout_conv is not None:
        x = Dropout(dropout_conv)(x)
    
    x = Flatten()(x)
    x = Dense(512, activation='relu',
                      W_regularizer=l2(0.01))(x)
    if dropout_dence is not None:
        x = Dropout(dropout_dence)(x)
    
    x = Dense(output_dim=2, 
              activation='softmax', 
              name='is_cancer')(x)
    return Model(inputs, x)

In [7]:
from keras.regularizers import l1, l2

def cancer_clf(input_chanels=10,
               shape=(4, 3, 3, 256),
               dropout_conv=.2,
               dropout_dence=.2):
   
    # Determine proper input shape
    if K.image_dim_ordering() != 'tf':
        print('Wrong dim ordering: should be TF')
        
    inputs = [Input(shape=shape) 
              for i in range(input_chanels)]
    
    interim = [Flatten(),
               Dropout(dropout_dence),
               Dense(256, activation='relu',
                     W_regularizer=l2(0.01)),
              ]
    
    x = [apply(interim, tensor) 
         for tensor in inputs]
    
    x = merge(x, mode='concat')
    
    x = Dropout(dropout_dence)(x)

    x = Dense(256, activation='relu',
        W_regularizer=l2(0.01))(x)
    
    x = Dropout(dropout_dence)(x)
    
    x = Dense(output_dim=2, 
              activation='softmax', 
              name='is_cancer')(x)

    return Model(inputs, x)

In [8]:
def train_test_split(savef=False, prefix='BN'):
    cancer_files = glob(join(path, 'CANCER', 
                             '*' + prefix + '*'))
    bening_files = glob(join(path, 'BENING', 
                             '*' + prefix + '*'))
    patients_cancer = set([basename(patch)[:32] for patch in cancer_files])
    patients_bening = set([basename(patch)[:32] for patch in bening_files])

    patients_cancer = list(patients_cancer)
    random.shuffle(patients_cancer)
    patients_bening = list(patients_bening)
    random.shuffle(patients_bening)

    train = patients_bening[int(len(patients_bening) * TRAIN_VAL_RAIT):] 
    train += patients_cancer[int(len(patients_cancer) * TRAIN_VAL_RAIT):]
    valid = patients_bening[:int(len(patients_bening) * TRAIN_VAL_RAIT)] 
    valid += patients_cancer[:int(len(patients_cancer) * TRAIN_VAL_RAIT)]

    train = [path for path in cancer_files + bening_files 
             if basename(path)[:32] in train]
    valid = [path for path in cancer_files + bening_files 
             if basename(path)[:32] in valid]
    if savef:
        pickle.dump(train, 
                    open(join(PATH['WEIGHTS'], 'train_files_cancer_clf'), 'wb'))
        pickle.dump(valid, 
                    open(join(PATH['WEIGHTS'], 'valid_files_cancer_clf'), 'wb'))
    return train, valid

In [9]:
# train, valid = train_test_split(savef=True)

In [10]:
train = pickle.load(open(join(PATH['WEIGHTS'], 'train_files_cancer_clf'), 'rb'))
valid = pickle.load(open(join(PATH['WEIGHTS'], 'valid_files_cancer_clf'), 'rb'))

In [24]:
def batch_generator(patch_paths, 
                    channels=9, 
                    batch_size=32,
                    augment=False,
                    CPU=24):
    
    counter = 0
    if augment:
        random.shuffle(patch_paths)
    number_of_batches = ceil(len(patch_paths) 
                             / batch_size)
    
    while True:
        batch_files = patch_paths[batch_size * counter:
                                  batch_size * (counter + 1)]
        
        with Pool(CPU) as pool:
            patch_list = pool.map(load, batch_files) 
        
        
        counter += 1
        labels = [[1, 0] if 'CANCER' in patch_path else [0, 1] 
                  for patch_path in batch_files]

        mask = asarray([True] * channels + [False] * (FULL_CHANNELS_N - channels))
        
        for i, patient in enumerate(patch_list):
            if augment:
                random.shuffle(mask)
            patch_list[i] = patient[mask]
            if augment:
                random.shuffle(patch_list[i])
                    
        patch_list = swapaxes(asarray(patch_list), 0, 1)
        patch_list = [nodule for nodule in patch_list]
        yield patch_list, asarray(labels)
        
        if counter == number_of_batches:
            if augment:
                random.shuffle(patch_paths)
            counter = 0

In [25]:
model = cancer_clf(input_chanels=7, dropout_conv=.2, dropout_dence=.2)
model.compile('adam', 'categorical_crossentropy')
# model.load_weights(join(PATH['WEIGHTS'], '3D_cancer_cancer'))

In [26]:
train_generator = batch_generator(train, batch_size=64, channels=7, augment=True)
valid_generator = batch_generator(valid, batch_size=64, channels=7, augment=False)

In [27]:
checkpoint = ModelCheckpoint(filepath=join(PATH['WEIGHTS'], '3D_cancer_cancer_a'), 
                             verbose=1, 
                             save_best_only=True)

history = model.fit_generator(train_generator, 
                              samples_per_epoch=(len(train) // BATCH_SIZE) * BATCH_SIZE , 
                              nb_epoch=100, 
                              callbacks=[checkpoint], 
                              validation_data=valid_generator,  
                              nb_val_samples=len(valid),
                              nb_worker=1)
    
model.save_weights(join(PATH['WEIGHTS'], '3D_cancer_cancer_finala'))

Epoch 1/100
29248/29280 [============================>.] - ETA: 0s - loss: 0.9083

/home/a.dobrenkii/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 0.61022, saving model to /home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/WEIGHTS/3D_cancer_cancer_a
29310/29280 [==============================] - 667s - loss: 0.9077 - val_loss: 0.6102
Epoch 2/100
29310/29280 [==============================] - 662s - loss: 0.5856 - val_loss: 0.6075
Epoch 3/100
29310/29280 [==============================] - 692s - loss: 0.5750 - val_loss: 0.5950
Epoch 4/100
29310/29280 [==============================] - 729s - loss: 0.5664 - val_loss: 0.6028
Epoch 5/100
29310/29280 [==============================] - 728s - loss: 0.5640 - val_loss: 0.6638
Epoch 6/100
29310/29280 [==============================] - 721s - loss: 0.5596 - val_loss: 0.6279
Epoch 7/100
29310/29280 [==============================] - 744s - loss: 0.5519 - val_loss: 0.6526
Epoch 8/100
29310/29280 [==============================] - 731s - loss: 0.5506 - val_loss: 0.6700
Epoch 9/100
29310/29280 [==============================] - 701s - loss: 0.

Process ForkPoolWorker-486016:
Process ForkPoolWorker-486020:
Process ForkPoolWorker-486015:
Process ForkPoolWorker-486013:
Process ForkPoolWorker-486018:
Process ForkPoolWorker-486023:
Process ForkPoolWorker-486031:
Process ForkPoolWorker-486030:
Process ForkPoolWorker-486014:
Process ForkPoolWorker-486017:
Process ForkPoolWorker-486032:
Process ForkPoolWorker-486033:
Process ForkPoolWorker-486021:
Process ForkPoolWorker-486019:
Process ForkPoolWorker-486026:
Process ForkPoolWorker-486024:
Process ForkPoolWorker-486022:
Process ForkPoolWorker-486029:
Process ForkPoolWorker-486028:
Process ForkPoolWorker-486025:
Process ForkPoolWorker-486027:
Process ForkPoolWorker-486034:
Process ForkPoolWorker-486035:
Process ForkPoolWorker-486036:
Traceback (most recent call last):
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self

KeyboardInterrupt: 

  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
KeyboardInterrupt
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/multiprocessin

In [28]:
model.save_weights(join(PATH['WEIGHTS'], '3D_cancer_cancer_finala'))